# Imports

In [63]:
import spacy
from spacy.tokens.doc import Doc
import pandas as pd
from collections import Counter, defaultdict
# from sets import Frozenset
from datetime import datetime

ImportError: cannot import name Frozenset

In [4]:
nlp = spacy.load('en')

In [5]:
test_loc = 'test_dump.bin'
train_loc = 'train_dump.bin'

# Read data

In [7]:
train = pd.read_csv('../data/train.csv').fillna('')
test = pd.read_csv('../data/test.csv').fillna('')
print train.shape, test.shape

(404290, 6) (2345796, 3)


# Write training questions
In the order they appear, not the order of their ids

In [46]:
# with open(train_loc, 'wb') as file_:
#     for idx, question in enumerate(train_array):
#         if idx % 10000 == 0: print(idx, question)
#         file_.write(nlp(question.decode('utf-8')).to_bytes())
with open(train_loc, 'wb') as file_:
    for i in range(len(train)):
        f = train.loc[i]
        if i % 10000 == 0: print i, f['question2']
        file_.write(nlp(f['question1'].decode('utf-8')).to_bytes())
        file_.write(nlp(f['question2'].decode('utf-8')).to_bytes())

0 What is the step by step guide to invest in share market?
10000 Is it safe for a woman to travel alone in Japan?
20000 What triggers you the most when you play video games?
30000 Is domino's pizza halal?
40000 How should I start learning Python for Data Science?
50000 Why is cloning considered unethical?
60000 How do you get rid of a virus on an iPhone?
70000 What are some teenage cake ideas?
80000 How do I recover deleted files on an Android phone or tablet?
90000 Which are the best Hollywood movies of 2016?
100000 What is the process of getting a surgical residency in UK after completing MBBS from India?
110000 How do I become an entrepreneur?
120000 How can I become a billionaire?
130000 What is it like to grow up in Japan, SIngapore and Hong Kong?
140000 Which is a good car in the range of 2-3lacs on Delhi road?
150000 How do I grow my hair really fast and easiest way?
160000 What are the career options after graduating with a B.A. in philosophy?
170000 Is welfare a citizenship r

# Read training data
Elements $(2n, 2n+1)$ are the questions for pair $n$

In [6]:
docs = []
i = 0
print datetime.now().strftime('%Y-%m-%d %H:%M:%S')
with open(train_loc, 'rb') as file_:
    for byte_string in Doc.read_bytes(file_):
        if i%10000 == 0: print i, datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        docs.append(Doc(nlp.vocab).from_bytes(byte_string))
        i += 1
#         if i == 10: break
print datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print(type(docs[0]))

2017-05-14 05:21:53
0 2017-05-14 05:21:53
10000 2017-05-14 05:21:58
20000 2017-05-14 05:21:59
30000 2017-05-14 05:22:01
40000 2017-05-14 05:22:02
50000 2017-05-14 05:22:03
60000 2017-05-14 05:22:05
70000 2017-05-14 05:22:07
80000 2017-05-14 05:22:09
90000 2017-05-14 05:22:11
100000 2017-05-14 05:22:13
110000 2017-05-14 05:22:15
120000 2017-05-14 05:22:16
130000 2017-05-14 05:22:18
140000 2017-05-14 05:22:19
150000 2017-05-14 05:22:20
160000 2017-05-14 05:22:22
170000 2017-05-14 05:22:24
180000 2017-05-14 05:22:27
190000 2017-05-14 05:22:29
200000 2017-05-14 05:22:32
210000 2017-05-14 05:22:34
220000 2017-05-14 05:22:35
230000 2017-05-14 05:22:36
240000 2017-05-14 05:22:38
250000 2017-05-14 05:22:39
260000 2017-05-14 05:22:41
270000 2017-05-14 05:22:42
280000 2017-05-14 05:22:43
290000 2017-05-14 05:22:45
300000 2017-05-14 05:22:46
310000 2017-05-14 05:22:47
320000 2017-05-14 05:22:48
330000 2017-05-14 05:22:50
340000 2017-05-14 05:22:51
350000 2017-05-14 05:22:53
360000 2017-05-14 05:2

In [8]:
# print(docs[23986])
# for word in docs[23986]:
#     print(word.lemma_)
    
d = docs[23990]
print([(word.orth_, word.lemma_) for word in d])
list(d.ents)
# print(Counter([(token.orth_, token.pos_, token.lemma_) for token in d]))

[(u'I', u'-PRON-'), (u'am', u'be'), (u'financially', u'financially'), (u'stuck', u'stuck'), (u'in', u'in'), (u'a', u'a'), (u'half', u'half'), (u'baked', u'baked'), (u'relationship', u'relationship'), (u'.', u'.'), (u'Can', u'can'), (u'someone', u'someone'), (u'help', u'help'), (u',', u','), (u'how', u'how'), (u'can', u'can'), (u'I', u'-PRON-'), (u'get', u'get'), (u'out', u'out'), (u'of', u'of'), (u'this', u'this'), (u'?', u'?')]


[]

In [54]:
def covertPropn(pos):
    if pos == u'PROPN':
        return u'NOUN'
    else: return pos

In [79]:
for i in range(0,len(docs),2):
    fv = {}
    d1 = docs[i]
    d2 = docs[i+1]

    s1 = set(list(str(e) for e in d1.ents))
    s2 = set(list(str(e) for e in d2.ents))
    
    sameents = len(s1 & s2)
    diffents = len(s1 ^ s2)
    fv['sameents'] = sameents
    fv['diffents'] = diffents
    
    s1 = set([(covertPropn(word.pos_), word.lemma_) for word in d1 if not word.is_stop])
    s2 = set([(covertPropn(word.pos_), word.lemma_ ) for word in d2 if not word.is_stop])
    diff = s1 ^ s2
    same =  s1 & s2

    samenum = len(same)
    same = Counter([t[0] for t in same])
    fv.update(same)
    fv['samenum'] = samenum

    diffnum = len(diff)
    diff = Counter([t[0] + '_d' for t in diff])
    fv['diffnum'] = diffnum
    fv.update(diff)
    
    print(d1, d2)
    print fv
    if i > 100: break


(What is the step by step guide to invest in share market in india?, What is the step by step guide to invest in share market?)
{'sameents': 0, u'NOUN': 4, 'samenum': 6, 'diffents': 0, 'diffnum': 1, u'PUNCT': 1, u'VERB': 1, u'NOUN_d': 1}
(What is the story of Kohinoor (Koh-i-Noor) Diamond?, What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?)
{'sameents': 2, u'NOUN': 4, 'samenum': 8, u'ADJ_d': 1, u'VERB_d': 2, 'diffents': 1, 'diffnum': 5, u'PUNCT': 4, u'NOUN_d': 2}
(How can I increase the speed of my internet connection while using a VPN?, How can Internet speed be increased by hacking through DNS?)
{'sameents': 0, u'PRON_d': 1, u'NOUN': 1, 'samenum': 3, u'VERB_d': 1, 'diffents': 2, 'diffnum': 6, u'PUNCT': 1, u'VERB': 1, u'NOUN_d': 4}
(Why am I mentally very lonely? How can I solve it?, Find the remainder when [math]23^{24}[/math] is divided by 24,23?)
{'sameents': 0, u'ADV_d': 1, 'samenum': 1, u'ADJ_d': 1, u'VERB_d': 3, u'PUNCT_d': 2, u'NUM_d': 1, 

In [46]:
Counter(e2)

Counter({'Capricorn': 2, 'Moon': 1, 'Sun': 1})

In [35]:
type(

spacy.tokens.span.Span

# Write test questions

In [54]:
print len(test)
with open(test_loc, 'wb') as file_:
    for i in range(len(test)):
        if i % 10000 == 0: print i
        f = test.loc[i]
        file_.write(nlp(f['question1'].decode('utf-8')).to_bytes())
        file_.write(nlp(f['question2'].decode('utf-8')).to_bytes())

NameError: name 'test' is not defined

# Read test questions

In [7]:
test_docs = []
i = 0
with open(test_loc, 'rb') as file_:
    for byte_string in Doc.read_bytes(file_):
        test_docs.append(Doc(nlp.vocab).from_bytes(byte_string))
        i += 1
print(type(test_docs[0]))

<type 'spacy.tokens.doc.Doc'>


In [ ]:
test_docs[1345288]

In [ ]:
testloc = 'test_serialize.bin'
with open(testloc, 'wb') as file_:
    file_.write(nlp(u'This is a document.').to_bytes())
    file_.write(nlp(u'This is another.').to_bytes())

In [7]:
docs = []
with open(testloc, 'rb') as file_:
    for byte_string in Doc.read_bytes(file_):
        docs.append(Doc(nlp.vocab).from_bytes(byte_string))
assert len(docs) == 2

In [8]:
p = docs[0]

In [12]:
bow = []
for word in p:
#     if word.is_stop: continue
    bow.append(word.lemma_)
Counter(bow)

Counter({u'.': 1, u'a': 1, u'be': 1, u'document': 1, u'this': 1})

In [17]:
print(test.loc[20])

test_id                                                     20
question1    How matter at MIT? Will performing poorly in 1...
question2    I have passed 5 AP tests with scores trump 5. ...
Name: 20, dtype: object


In [24]:
doc = docs[24890]
print doc
print(' '.join(word.lemma_ for word in doc))

What are some good books to read about Islamic philosophy?
what be some good book to read about islamic philosophy ?


In [25]:
lemmas = []
for idx, doc in enumerate(docs):
    lemma = []
    for word in doc:
        if not word.is_punct: lemma.append(word.lemma_)
    lemmas.append(lemma)
    if idx == 20: break

In [ ]:
print(type(lemmas[0][0]))